# Contamination Detector Module MVP

I have tried detecting potentially contaminated segments in our training data.

According to the seminar and project scope we have performed two main scopoes.
1. **Reference Benchmark Comparison:** Each segment is compared against a reference set (synthetic here) via semantic similarity.
2. **PaCoST-Inspired Confidence Testing:** For each segment, we generate a perturbed variant and compute model confidence (via perplexity) using a lightweight LM. A significant gap in confidence (lower perplexity on the original) indicates possible memorization or contamination.

In [1]:
# Imports
import os
import random
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from scipy.stats import ttest_rel

# For timing and progress display
from tqdm.notebook import tqdm

## Loading the preprocessed data

In [2]:
data_file = os.path.join("data", "preprocessed_wikitext103_subset.csv")
df = pd.read_csv(data_file, on_bad_lines='skip', engine='python')
print("Loaded data shape:", df.shape)
print("Sample rows:")
print(df.head())

Loaded data shape: (186070, 5)
Sample rows:
                                                text  text_length  \
0  = Valkyria Chronicles III =\nSenjō no Valkyria...        20297   
1  = Valkyria Chronicles III =\nSenjō no Valkyria...        20297   
2  = Valkyria Chronicles III =\nSenjō no Valkyria...        20297   
3  = Valkyria Chronicles III =\nSenjō no Valkyria...        20297   
4  = Valkyria Chronicles III =\nSenjō no Valkyria...        20297   

                                        cleaned_text  \
0  = valkyria chronicles iii = senj no valkyria 3...   
1  = valkyria chronicles iii = senj no valkyria 3...   
2  = valkyria chronicles iii = senj no valkyria 3...   
3  = valkyria chronicles iii = senj no valkyria 3...   
4  = valkyria chronicles iii = senj no valkyria 3...   

                                              tokens  \
0  ['=', 'val', '##ky', '##ria', 'chronicles', 'i...   
1  ['=', 'val', '##ky', '##ria', 'chronicles', 'i...   
2  ['=', 'val', '##ky', '##ria', 'ch

### Load/Define Reference Benchmark Data
For now, I am defining a small reference set, later I plan to add texts from projectGutenberg or alike.

In [3]:
reference_texts = [
    "this is a known contaminated text from benchmark dataset",
    "another reference text that should not be in the training data",
    "benchmark evaluation text that must remain separate"
]

# Load a SentenceTransformer model for computing embeddings
ref_model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for the reference texts
ref_embeddings = ref_model.encode(reference_texts, convert_to_tensor=True)

## Define Functions for reference comparison

a function that computes maximum cosine similarity between a given segment and the reference texts. If the similarity exceeds a threshold (e.g., 0.9), we flag the segment as potentially contaminated by reference overlap.

In [4]:
def check_reference_similarity(segment, ref_embeddings, threshold=0.9):
    # Compute embedding for the segment
    segment_embedding = ref_model.encode(segment, convert_to_tensor=True)
    # Compute cosine similarities with each reference embedding
    cos_scores = util.cos_sim(segment_embedding, ref_embeddings)
    max_sim = cos_scores.max().item()
    flag = max_sim >= threshold
    return max_sim, flag

## Confidence Testing(PaCoST-Inspired)

**Perturbation Function:** We define a simple perturbation function that shuffles the words in the segment. (In practice, you might use more advanced paraphrasing techniques.)

**Perplexity Calculation:** We use a lightweight LM (DistilGPT-2) to compute the perplexity of a text. Lower perplexity indicates higher confidence.

**Note:** Perplexity is computed as the exponential of the loss from the model. I have used small batch size for avoiding memory issues as most of the code is working locally.


In [5]:
def perturb_text(text):
    words = text.split()
    if len(words) <= 1:
        return text  # No perturbation possible for single-word texts
    random.shuffle(words)
    return " ".join(words)

# Load a lightweight LM for perplexity calculation
lm_model_name = "distilgpt2"  # a lightweight model
lm_model = AutoModelForCausalLM.from_pretrained(lm_model_name)
lm_tokenizer = AutoTokenizer.from_pretrained(lm_model_name)
lm_model.eval()
if torch.cuda.is_available():
    lm_model.to("cuda")

def compute_perplexity(text, model, tokenizer):
    # Encode the text and compute loss (per-token)
    encodings = tokenizer(text, return_tensors='pt')
    if torch.cuda.is_available():
        encodings = {k: v.to("cuda") for k, v in encodings.items()}
    with torch.no_grad():
        outputs = model(**encodings, labels=encodings["input_ids"])
    loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

## Contamination detection on a Batch of Segments

processing the data in batches:
1. Check reference similarity
2. Compute perplexity for the original segment.
3. Generate a perturbed version and compute its perplexity.
4. Flag the segment if:
    - The max reference similarity is above a threshold (0.8), OR
    - The original perplexity is significantly lower than the perturbed perplexity
- **Simple Ration Test:** if perplexity(original) < 0.8 * perplexity(perturbed), I flag it.

In [6]:
sample_df = df.sample(n=1000, random_state=42).copy()

# lists to hold the results
ref_similarities = []
ref_flags = []
perplexity_orig_list = []
perplexity_perturbed_list = []
confidence_flags = []

# threshold for perplexity ratio
perplexity_ratio_threshold = 0.8

In [7]:
# Process each segment in the sample
for seg in tqdm(sample_df['segments'].tolist(), desc="Processing segments"):
    # Reference similarity check
    max_sim, ref_flag = check_reference_similarity(seg, ref_embeddings, threshold=0.9)
    ref_similarities.append(max_sim)
    ref_flags.append(ref_flag)

    # Compute perplexity for original
    try:
        ppl_orig = compute_perplexity(seg, lm_model, lm_tokenizer)
    except Exception as e:
        ppl_orig = None
    # Generate perturbed version and compute perplexity
    perturbed_seg = perturb_text(seg)
    try:
        ppl_perturbed = compute_perplexity(perturbed_seg, lm_model, lm_tokenizer)
    except Exception as e:
        ppl_perturbed = None

    perplexity_orig_list.append(ppl_orig)
    perplexity_perturbed_list.append(ppl_perturbed)

    # Check if confidence gap flag condition is met (if both perplexities are computed)
    if (ppl_orig is not None and ppl_perturbed is not None and ppl_perturbed > 0):
        conf_flag = ppl_orig < perplexity_ratio_threshold * ppl_perturbed
    else:
        conf_flag = False
    confidence_flags.append(conf_flag)

# Add the results to the sample DataFrame
sample_df['ref_similarity'] = ref_similarities
sample_df['ref_flag'] = ref_flags
sample_df['ppl_original'] = perplexity_orig_list
sample_df['ppl_perturbed'] = perplexity_perturbed_list
sample_df['confidence_flag'] = confidence_flags

# Combine flags: flag if either reference flag or confidence flag is True
sample_df['contamination_flag'] = sample_df['ref_flag'] | sample_df['confidence_flag']

Processing segments:   0%|          | 0/1000 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [9]:
sample_df.head()

,text,text_length,cleaned_text,tokens,segments,ref_similarity,ref_flag,ppl_original,ppl_perturbed,confidence_flag,contamination_flag
151849,= William Wilberforce =\nWilliam Wilberforce (...,51825,= william wilberforce = william wilberforce (2...,"['=', 'william', 'wil', '##ber', '##force', '=...",wilberforce's house in old palace yard became ...,-0.010654,False,240.796692,4236.520996,True,True
31078,= Roger Federer =\nRoger Federer (German: [ˈfe...,56964,= roger federer = roger federer (german: [ fe ...,"['=', 'roger', 'federer', '=', 'roger', 'feder...",= = = federer vs. agassi = = = federer and aga...,0.133301,False,83.830208,841.818359,True,True
120548,= USS Yancey (AKA-93) =\nUSS Yancey (AKA-93 / ...,20321,= uss yancey (aka-93) = uss yancey (aka-93 / l...,"['=', 'uss', 'yan', '##cey', '(', 'aka', '-', ...",yancey entered tokyo bay on the morning of 2 s...,0.030656,False,174.029938,1052.760254,True,True
117870,= Alberta and Great Waterways Railway scandal ...,28416,= alberta and great waterways railway scandal ...,"['=', 'alberta', 'and', 'great', 'waterways', ...","guarantees to the so-called ""a & gw"" were more...",0.034958,False,82.243286,1485.180176,True,True
44726,= Gregory Helms =\nGregory Shane Helms (born J...,24202,= gregory helms = gregory shane helms (born ju...,"['=', 'gregory', 'helm', '##s', '=', 'gregory'...",helms and storm got a shot at the wcw tag team...,0.133794,False,365.937317,1935.085693,True,True


## Saving Flagged segments

summary stats and saving the flagged segments

In [8]:
# Count the number of flagged segments
num_flagged = sample_df['contamination_flag'].sum()
print(f"Number of flagged segments in the sample: {num_flagged} / {len(sample_df)}")

# Save the results for further inspection
output_dir = "data"
os.makedirs(output_dir, exist_ok=True)
flagged_output_file = os.path.join(output_dir, "contamination_flags_sample.csv")
sample_df.to_csv(flagged_output_file, index=False)
print(f"Flagged segments saved to {flagged_output_file}")

Number of flagged segments in the sample: 988 / 1000
Flagged segments saved to data/contamination_flags_sample.csv
